In [31]:
import os
import glob as glob
import pandas as pd
import numpy as np
from events_prep import *

In [73]:
data_dir = '/home/finc/Dropbox/Projects/SelfReg/data/behavioral/raw/'
out_dir = '/home/finc/Dropbox/Projects/SelfReg/data/behavioral/prep/'

files = np.sort(os.listdir(data_dir))

for sub in files:
    file_path = f'{data_dir}{sub}/{sub}_surveyMedley.csv'
    if not os.path.exists(file_path):
        print(f'Survey data not exist for sub-{sub}')
    else:
        file_name = os.path.basename(file_path)
        df = pd.read_csv(file_path)
        
        start_time = df.query('trial_id == "fmri_trigger_wait"').iloc[-1]['time_elapsed']
        df.time_elapsed -= start_time
        
        # correct start time for problematic scans
        df.time_elapsed -= get_timing_correction(file_name)
        df_clean = create_survey_event(df)
        df_clean = df_clean.dropna(axis=0).reset_index(drop=True)
        df_clean['trial_type'] = df_clean['item_text'].map(get_items_order())
        df_clean['trial_no'] = np.arange(1,41)
        
        events = pd.DataFrame(df_clean, columns=['onset', 'trial_type'])
        events['duration'] = 3.982
        
        new_dir = f'{out_dir}sub-{sub}'
        if not os.path.exists(new_dir):
            os.makedirs(new_dir)
            
        events.to_csv(f'{new_dir}/sub-{sub}_surveyMedley_events.csv', index = False)
        df_clean.to_csv(f'{new_dir}/sub-{sub}_surveyMedley_clean.csv', index = False)

Survey data not exist for sub-s144
Survey data not exist for sub-s533
Survey data not exist for sub-s600
Survey data not exist for sub-s603
Survey data not exist for sub-s620
Survey data not exist for sub-s639
Survey data not exist for sub-s640


In [53]:
f'{out_dir}{sub}'

'/home/finc/Dropbox/Projects/SelfReg/data/behavioral/events/s061'

tests
------

In [59]:
sub = 's358'
file_path = glob.glob(f'{data_dir}{sub}/*surveyMedley.csv')[0]
file_name = os.path.basename(file_path)

df = pd.read_csv(file_path)
df.columns

start_time = df.query('trial_id == "fmri_trigger_wait"').iloc[-1]['time_elapsed']
df.time_elapsed-=start_time
# correct start time for problematic scans
df.time_elapsed-=get_timing_correction(file_name)
df_clean = create_survey_event(df)
df_clean = df_clean.dropna(axis=0).reset_index(drop=True)
df_clean['trial_type'] = df_clean['item_text'].map(get_items_order())
df_clean['trial_no'] = np.arange(1,41)

df_clean.shape


(40, 14)

In [71]:
events = pd.DataFrame(df_clean, columns=['onset', 'duration', 'trial_type'])
#events.to_csv(f'sub-{sub}_surveyMedley_events.csv', index = False)
np.median(df_clean['rt'].iloc[df_clean['rt'].values>0].values)/1000

3.5675